In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib
import os
import polars as pl
from concurrent.futures import as_completed, ProcessPoolExecutor


In [3]:
!pip install schrodinger openbabel rdkit joblib

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 1.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 44.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 83.2 MB/s eta 0:00:00:00:010:01
  Created wheel for schrodinger: filename=schrodinger-0.1-py3-none-any.whl size=4981 sha256=7126ddc4b564d64d0c404ba4da3f2db6be9dd9bdc220b2e0cf1213c9f6cd7025
  Stored in directory: /root/.cache/pip/wheels/ef/e1/5c/c8ce3a67a4a49a11a2e789c89adfde87476613af14fee4c961
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      Guessing Open Babel location:
      - include_dirs: ['/opt/conda/include/python3.10', '/usr/local/include/openbabel3']
      - library_dirs: ['/usr/local/lib']
      
      Error: SWIG failed. Is Open Babel installed?
      You may need to ma

In [ ]:
!conda install -c conda-forge -c schrodinger joblib openbabel rdkit pymol-bundle -y

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from catboost import CatBoostClassifier
from rdkit.Chem import rdMolDescriptors
import joblib

In [ ]:
test_file = '/kaggle/input/leash-BELKA/test.csv'

In [ ]:
with open('/kaggle/input/list-of-ojects/score_model_list.joblib', 'rb') as f:
    models = joblib.load(f)

In [ ]:
best_model = models[1][1]

In [ ]:
def add_attributes(smiles, brd4, hsa, seh, ii):

    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    
    # Calculate the number of atoms and bonds
    atom_count = mol.GetNumAtoms()
    bond_count = mol.GetNumBonds()

    # Calculate the molecular weight
    molecular_weight = Descriptors.ExactMolWt(mol)
    molecular_weight = round(molecular_weight, 2)
    
    
    # logP = rdMolDescriptors. MolLogP(mol)
    # logP = round(logP , 2)
    hb_acceptor = rdMolDescriptors.CalcNumHBA(mol)
    hb_donor = rdMolDescriptors.CalcNumHBD(mol)

    tpsa = rdMolDescriptors.CalcTPSA(mol)

    rotatable_bond = rdMolDescriptors.CalcNumRotatableBonds(mol)

    # Calculate the number of charges (positive and negative)
    try:
        charges = [atom.GetProp('_Charge') for atom in mol.GetAtoms()]
        pos_charges = len([c for c in charges if c > 0])
        neg_charges = len([c for c in charges if c < 0])

    except KeyError:
        pos_charges = 0
        neg_charges = 0

    return {
        'id':ii,
        'atoms': atom_count,
        'bonds': bond_count,
        'positive_charges': pos_charges,
        'negative_charges': neg_charges,
        'molecular_weight' : molecular_weight,
        # 'LogP' : logP ,
        'HBA' : hb_acceptor ,
        'HBD' : hb_donor,
        'TPSA' : tpsa ,
        'Rotatable Bonds' : rotatable_bond,
        'BRD4':brd4,
        'HSA':hsa,
        'sEH':seh,
#         'binds': label
    }

In [ ]:
import os
test_file = '/kaggle/input/leash-BELKA/test.csv'
# Process the test.parquet file chunk by chunk
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['HSA'] = 0
    df_test['BRD4'] = 0
    df_test['sEH'] = 0
    df_test.loc[df_test['protein_name']=='HSA', 'HSA'] = 1
    df_test.loc[df_test['protein_name']=='BRD4', 'BRD4'] = 1
    df_test.loc[df_test['protein_name']=='sEH', 'sEH'] = 1
    feats = []
    with ProcessPoolExecutor(max_workers=8) as pool:
        futures = [pool.submit(add_attributes, row['molecule_smiles'], row['BRD4'], row['HSA'], row['sEH'],
                                row['id']) 
                  for ii, row in df_test.iterrows()]
        for f in as_completed(futures):
            feats.append(f.result())
    df_X = pd.DataFrame(feats)
    
    probabilities = best_model.predict_proba(df_X[best_model.feature_names_])[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_X['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))